In [1]:
import gzip
import json
from pprint import pprint
import pandas as pd
import jsonlines
from collections import Counter
from mltb2.text import clean_all_invisible_chars_and_whitespaces

In [2]:
def select_data(data: dict):
    result = {}

    llm_response = data["llm_response"]

    # ignore uncomplete response
    if llm_response["finish_reason"] != "stop":
        return None

    result["imperative_formal"] = clean_all_invisible_chars_and_whitespaces(data["imperative_formal"])
    result["imperative_informal"] = clean_all_invisible_chars_and_whitespaces(llm_response["content"])

    return result

In [3]:
extracted_data = []

with gzip.open("./data/imp_to_informal_backup.jsonl.gz", "r") as gzip_in:
    jsonlines_reader = jsonlines.Reader(gzip_in)
    for data in jsonlines_reader:
        selected_data = select_data(data)
        if selected_data is not None:
            extracted_data.append(selected_data)

In [4]:
df = pd.DataFrame(extracted_data)
df

,imperative_formal,imperative_informal
0,"Erklären Sie, wo und wann Eugenia Gapchinska i...","Erkläre, wo und wann Eugenia Gapchinska ihr er..."
1,"Zeigen Sie auf, welche Rolle Joanna Pizoń-Świt...","Zeige auf, welche Rolle Joanna Pizoń-Świtkowsk..."
2,"Erklären Sie, wie das AN/AAR-54 mehrere Ziele ...","Erkläre, wie das AN/AAR-54 mehrere Ziele gleic..."
3,Identifizieren Sie den Vorläufer des FHP-Modells.,Identifiziere den Vorläufer des FHP-Modells.
4,Schildern Sie Manfred Kummers Rolle an der Tec...,Schildere Manfred Kummers Rolle an der Technis...
...,...,...
741714,Nennen Sie das Geburts- und Todesdatum von Pet...,Nenne das Geburts- und Todesdatum von Peter En...
741715,"Teilen Sie mit, warum Czernin seine diplomatis...","Teile mit, warum Czernin seine diplomatische L..."
741716,"Beschreiben Sie Hobes Tätigkeiten in Münster, ...","Beschreibe Hobes Tätigkeiten in Münster, Celle..."
741717,Beschreiben Sie das Klima in der Jakutischen A...,Beschreibe das Klima in der Jakutischen Autono...


In [5]:
# check if there are duplicates in imperative_formal
df[df["imperative_formal"].duplicated(keep=False)]

,imperative_formal,imperative_informal


## Insight

- there are no suplicate imperative_formal

In [6]:
# remove duplicates in imperative_informal
df.drop_duplicates("imperative_informal", inplace=True, keep=False)
df

,imperative_formal,imperative_informal
0,"Erklären Sie, wo und wann Eugenia Gapchinska i...","Erkläre, wo und wann Eugenia Gapchinska ihr er..."
1,"Zeigen Sie auf, welche Rolle Joanna Pizoń-Świt...","Zeige auf, welche Rolle Joanna Pizoń-Świtkowsk..."
2,"Erklären Sie, wie das AN/AAR-54 mehrere Ziele ...","Erkläre, wie das AN/AAR-54 mehrere Ziele gleic..."
3,Identifizieren Sie den Vorläufer des FHP-Modells.,Identifiziere den Vorläufer des FHP-Modells.
4,Schildern Sie Manfred Kummers Rolle an der Tec...,Schildere Manfred Kummers Rolle an der Technis...
...,...,...
741714,Nennen Sie das Geburts- und Todesdatum von Pet...,Nenne das Geburts- und Todesdatum von Peter En...
741715,"Teilen Sie mit, warum Czernin seine diplomatis...","Teile mit, warum Czernin seine diplomatische L..."
741716,"Beschreiben Sie Hobes Tätigkeiten in Münster, ...","Beschreibe Hobes Tätigkeiten in Münster, Celle..."
741717,Beschreiben Sie das Klima in der Jakutischen A...,Beschreibe das Klima in der Jakutischen Autono...


In [7]:
imperative_map = {k: v for k, v in zip(df["imperative_formal"], df["imperative_informal"])}

In [8]:
# load train data
with gzip.GzipFile("./data/wikipedia-22-12-de-train-data.json.gz", "r") as f:
    train_data = json.loads(f.read().decode('utf-8'))
assert len(train_data) == 129_591

In [9]:
# load test data
with gzip.GzipFile("./data/wikipedia-22-12-de-test-data.json.gz", "r") as f:
    test_data = json.loads(f.read().decode('utf-8'))
assert len(test_data) == 5_000

In [10]:
all_questions = []
all_imperatives = []

for d in train_data:
    if 'questions' in d:
        all_questions.extend(d['questions'])
    if 'imperatives' in d:
        all_imperatives.extend(d['imperatives'])

for d in test_data:
    if 'questions' in d:
        all_questions.extend(d['questions'])
    if 'imperatives' in d:
        all_imperatives.extend(d['imperatives'])

print("len all_questions", len(all_questions))
print("len all_imperatives", len(all_imperatives))

len all_questions 786353
len all_imperatives 793143


In [11]:
counter = Counter()
counter.update(all_questions)
counter.update(all_imperatives)
counter.update(df["imperative_informal"].tolist())

duplicates = set([item for item, count in counter.items() if count > 1])
len(duplicates)

1663

In [12]:
def add_imperative_informal(data: list):
    for d in data:
        # handle questions
        if "questions" in d:
            questions = d["questions"]
            del d["questions"]
            questions = [q for q in questions if q not in duplicates]  # remove duplicates
            if len(questions) > 0:
                d["question"] = questions

        # handle imperatives
        if "imperatives" in d:
            imperative_formals = d["imperatives"]
            del d["imperatives"]
            imperative_formals = [i for i in imperative_formals if i not in duplicates]  # remove duplicates
            if len(imperative_formals) > 0:
                new_imperative_informals = []
                for imperative_formal in imperative_formals:
                    if imperative_formal in imperative_map:
                        new_imperative_informal = imperative_map[imperative_formal]
                        if new_imperative_informal not in duplicates:
                            new_imperative_informals.append(new_imperative_informal)
                if len(new_imperative_informals) > 0:
                    d["imperative_informal"] = new_imperative_informals
                d["imperative_formal"] = imperative_formals
        assert "question" in d or "imperative_informal" in d or "imperative_formal" in d

In [13]:
add_imperative_informal(train_data)
add_imperative_informal(test_data)
assert len(train_data) == 129_591
assert len(test_data) == 5_000

In [14]:
train_data[0]

{'wiki_id': 1683529,
 'title': '2,4-Dinitrophenylhydrazin',
 'context': '2,4-Dinitrophenylhydrazin ist eine chemische Verbindung aus der Gruppe der aromatischen Hydrazine mit der Summenformel CHNO. Trockenes 2,4-Dinitrophenylhydrazin ist explosionsgefährlich; bei der Zersetzung entstehen nitrose Gase. Es wird mit Wasser phlegmatisiert, d. h., dem kommerziell erhältlichen Produkt werden ca. 0,5 ml Wasser pro Gramm DNPH zugesetzt, um die Substanz lager- und transportfähig zu machen (darum auch keine GHS-Gefahrstoffkenzeichnung als explosionsgefährlich). Es werden „erhebliche, negative Auswirkungen auf menschliche Gesundheit und Wasserökologie“ erwähnt. Diese eher subjektive und unbestimmte Aussage erlaubt jedoch keine Zuweisung von Gefahrenmerkmalen. Es liegt nur eine geringe akute Toxizität vor. Ähnlich wie andere als Düngemittel verwendete Stoffe, kann von negativen ökologischen Auswirkungen ausgegangen werden, wenn der Stoff ungewollt und in hoher Konzentration in die Umwelt gelangt.'

In [15]:
test_data[0]

{'wiki_id': 6019975,
 'title': 'A•DEvantgarde',
 'context': 'A•DEvantgarde ist ein Zusammenschluss von Komponisten in München. Er wurde 1991 auf Initiative Wilhelm Killmayers von Moritz Eggert und Sandeep Bhagwati als eingetragener Verein "aDEvantgarde – Projekte Neuer Musik" gegründet. Seitdem führt aDEvantgarde alle zwei Jahre das aDEvantgarde-Festival durch. Das "A•DEvantgarde-Festival" fand bisher im Wechsel mit der Münchener Biennale für Neues Musiktheater statt, die immer in geraden Jahren veranstaltet wurden, während das A•DEvantgarde-Festival in ungeraden Jahren stattfindet.',
 'context_uuid': 'd98662e3-64aa-472c-80a5-b744b4d22fc2',
 'question': ['Wann wurde der Verein "aDEvantgarde – Projekte Neuer Musik" gegründet und wer waren die Initiatoren?',
  'Was ist aDEvantgarde und wer sind seine Mitglieder?',
  'Wie oft findet das aDEvantgarde-Festival statt und in welchem Rhythmus wechselt es mit der Münchener Biennale ab?',
  'Wer hat den Verein aDEvantgarde ins Leben gerufen?',
 

In [16]:
# save train data
with gzip.GzipFile("./data/wikipedia-22-12-de-train-data-v2.json.gz", "w") as f:
    f.write(json.dumps(train_data, indent=1).encode("utf-8"))

In [17]:
# save test data
with gzip.GzipFile("./data/wikipedia-22-12-de-test-data-v2.json.gz", "w") as f:
    f.write(json.dumps(test_data, indent=1).encode("utf-8"))